# Generate a OneLake SAS token

## Authentication

This notebook uses interactive credentials which means it will open a browser window for interactive authentication.
Authentication behavior can be changed by altering the credential aquisition code below. Consult the [documentation](https://learn.microsoft.com/en-us/dotnet/api/overview/azure/identity-readme?view=azure-dotnet) for more details.

```csharp-interactive
var cred = new DefaultAzureCredential (includeInteractiveCredentials: true);
```

## Usage

To use this notebook, 

1. Replace the workspace id variable in the second cell with your workspace id.
1. Replace the path with the path to which you want to generate a SAS token. The path should start with the item id GUID.
1. If you want to share a directory, change the isDirectory variable to true.
1. Run the notebook
1. Copy the link below the second cell 




In [1]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Azure.Storage.Files.DataLake, 12.22.0"

using Azure.Core;
using Azure.Identity;
using Azure.Storage.Files.DataLake;
using Azure.Storage.Sas;

var cred = new DefaultAzureCredential (includeInteractiveCredentials: true);

Installed Packages Azure.Identity, 1.13.2 Azure.Storage.Files.DataLake, 12.22.0

In [ ]:
var workspaceid = "Guid of your workspace id";
var path = "Path to the file or folder you want to share. The path should begin with the item id  example {82cfff6e-9941-460c-a2ed-6981297d50da/Files/myfolder/myfile.json";
var isDirectory = false;
var permissions = DataLakeAccountSasPermissions.Read | DataLakeAccountSasPermissions.List;

var oneLakeUri = "https://onelake.dfs.fabric.microsoft.com/";
var client = new DataLakeServiceClient(new Uri(oneLakeUri), (TokenCredential)cred);

var delegationKey = await client.GetUserDelegationKeyAsync(DateTimeOffset.UtcNow.AddMinutes(-2), DateTimeOffset.UtcNow.AddMinutes(30));
var sasBuilder = new DataLakeSasBuilder
{
    FileSystemName = workspaceid,
    Path = path,
    StartsOn = DateTimeOffset.UtcNow.AddMinutes(-1),
    ExpiresOn = DateTimeOffset.UtcNow.AddMinutes(20),
    IsDirectory = isDirectory,
    Resource = isDirectory ? "d" : "b",
};

sasBuilder.SetPermissions(permissions);
var token = sasBuilder.ToSasQueryParameters(delegationKey, "onelake");

Console.WriteLine($"{oneLakeUri}{workspaceid}/{path}?{token}");
return;

Error: Azure.RequestFailedException: {"error":{"code":"PowerBINotAuthorizedException","pbi.error":{"code":"PowerBINotAuthorizedException","parameters":{},"details":[],"exceptionCulprit":1}}}
Status: 400 (Bad Request)
ErrorCode: BadRequest

Content:
{
  "error": {
    "code": "BadRequest",
    "message": "{\"error\":{\"code\":\"PowerBINotAuthorizedException\",\"pbi.error\":{\"code\":\"PowerBINotAuthorizedException\",\"parameters\":{},\"details\":[],\"exceptionCulprit\":1}}}",
    "target": null,
    "details": null
  }
}

Headers:
Server: Microsoft-HTTPAPI/2.0
Access-Control-Allow-Origin: *
Access-Control-Allow-Headers: REDACTED
Access-Control-Allow-Methods: REDACTED
Access-Control-Expose-Headers: REDACTED
Strict-Transport-Security: REDACTED
X-Content-Type-Options: REDACTED
x-ms-root-activity-id: REDACTED
Date: Fri, 04 Apr 2025 05:33:24 GMT
Content-Length: 283
Content-Type: application/json; charset=utf-8

   at Azure.Storage.Blobs.ServiceRestClient.GetUserDel